In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms, datasets, models
import torch.nn.functional as F

In [67]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

In [0]:
batch_size = 16
num_epochs = 5

In [0]:
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size,
                                          shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size,
                                         shuffle=False)

In [82]:
iterator = iter(trainloader)
img = next(iterator)
img[0].shape

torch.Size([16, 1, 28, 28])

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4* 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def flatten(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x




In [0]:
net = Net().to(device)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [0]:

min_test_loss = 1000

In [103]:
for epoch in range(num_epochs):
    running_loss_train = 0.0
    running_loss_test = 0.0
    
    net.train()
    
    for i,data in enumerate(trainloader,0):

      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      output = net(inputs)
      loss = criterion(output, labels)

      loss.backward()
      optimizer.step()

      running_loss_train += loss.item()      

    net.eval()

    correct_preds = 0
    total_preds = 0
    
    for i,data in enumerate(testloader,0):

      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      output = net(inputs)
      loss = criterion(output, labels)


      running_loss_test += loss.item()

      _, pred = torch.max(output.data, 1)
      correct_preds += (pred == labels).sum()
      total_preds += labels.size(0)
        
    accuracy = 100.00 * float(correct_preds)/float(total_preds)
    

        
    print(f'Epoch : {epoch+1}, Train Loss : {running_loss_train} Test Loss : {running_loss_test}, Accuracy : {accuracy}')

    
    if running_loss_test<min_test_loss:
        torch.save(net.state_dict(), 'mnist.pt')
        min_test_loss = running_loss_test
 

Epoch : 1, Train Loss : 17.16972103714943 Test Loss : 28.214121639728546, Accuracy : 98.93
Epoch : 2, Train Loss : 17.304844975471497 Test Loss : 25.160099029541016, Accuracy : 99.04
Epoch : 3, Train Loss : 12.407264158129692 Test Loss : 29.142756938934326, Accuracy : 98.95
Epoch : 4, Train Loss : 10.60317613184452 Test Loss : 27.478791311383247, Accuracy : 98.97
Epoch : 5, Train Loss : 9.1339810192585 Test Loss : 27.908689454197884, Accuracy : 98.96


In [109]:
net.load_state_dict(torch.load('mnist.pt'))


<All keys matched successfully>

In [0]:
'''The following part of the code has been taken entirely from - 
https://github.com/harveyslash/Facial-Similarity-with-Siamese-Networks-in-Pytorch '''

In [0]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [0]:
class ContrastiveLoss(torch.nn.Module):    

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [0]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(1, 4, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(4),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),


            nn.ReflectionPad2d(1),
            nn.Conv2d(8, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [0]:
Siamese = SiameseNetwork(net)
criterion = ContrastiveLoss()
optimizer = optim.Adam(Net.parameters(),lr = 0.001)